# Filtering the Noise: ML for Trustworthy Location Reviews

**Team:** OIIA OIIA 
**Date:** August 31, 2025  
**Challenge:** Design and implement an ML-based system to evaluate the quality and relevancy of Google location reviews

### Problem Statement
- **Gauge review quality**: Detect spam, advertisements*, irrelevant content*, and rants*
- **Assess relevancy**: Determine if review content is genuinely related to the location
- **Enforce policies**: Automatically flag reviews violating predefined policies

## 🔨 Setup

In [15]:
# ⚠️ Run this cell only if fresh runtime or first time setup

# Install required packages
%pip install transformers torch datasets pandas numpy scikit-learn matplotlib seaborn plotly
%pip install huggingface-hub accelerate
%pip install nltk spacy wordcloud
%pip install kaggle
%pip install textstat
print("All packages installed successfully!")

^C
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
All packages installed successfully!



[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# ⚠️ Run this cell only if fresh runtime or first time setup

# Import essential libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# NLP and ML libraries
import nltk
import spacy
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

# Data processing
import re
import string
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# Terminal commands
import os
from pathlib import Path
import shutil

# Download NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('vader_lexicon')

print("All imports successful!")

C:\Users\seanh\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


All imports successful!


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\seanh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\seanh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\seanh\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [ ]:
# ⚠️ Run this cell only if fresh runtime or first time setup

from kaggle.api.kaggle_api_extended import KaggleApi

# Kaggle API Setup & Downloading of Dataset to ./kaggle_data directory
def config_kaggle_api_token():
    # kaggle_dir = Path.home() / '.config' / 'kaggle'
    kaggle_dir = Path.home() / '.kaggle'
    kaggle_dir.mkdir(exist_ok=True)

    shutil.copy('./kaggle.json', kaggle_dir / 'kaggle.json')
    os.chmod(kaggle_dir / 'kaggle.json', 0o600)

def download_kaggle_dataset(path='./kaggle_data', dataset_name="denizbilginn/google-maps-restaurant-reviews"):
    api = KaggleApi()
    api.authenticate()
    dataset_name="denizbilginn/google-maps-restaurant-reviews"
    api.dataset_download_files(dataset_name,
                            path=path,
                            unzip=True)

## 📊 Data Collection & Loading

We'll use the provided Google Local Reviews dataset. You can also supplement with additional data sources.

In [ ]:
# ⚠️ Run this cell only if fresh runtime or first time setup

# Download Kaggle Dataset
config_kaggle_api_token()
download_kaggle_dataset()

Dataset URL: https://www.kaggle.com/datasets/denizbilginn/google-maps-restaurant-reviews


In [ ]:
# Data Loading Functions

def load_dataset(file_path):
    """Load dataset from local CSV file"""
    try:
        if os.path.exists(file_path):
            df = pd.read_csv(file_path)
            print(f"✅ Loaded {len(df)} rows from {file_path}")
            # df = standardize_columns(df)
            return df
        else:
            print(f"❌ File not found: {file_path}")
            return None
    except Exception as e:
        print(f"❌ Error loading local file: {e}")
        return None

def standardize_columns(df):
    """Standardize column names to match our expected format"""
    # Common column mappings
    column_mappings = {
        'text': 'review_text',
        'review': 'review_text',
        'comment': 'review_text',
        'content': 'review_text',
        'review_text': 'review_text',

        'rating': 'rating',
        'stars': 'rating',
        'score': 'rating',
        'star_rating': 'rating',

        'business': 'business_name',
        'restaurant': 'business_name',
        'place_name': 'business_name',
        'name': 'business_name',

        'user': 'user_id',
        'user_name': 'user_id',
        'reviewer': 'user_id',

        'date': 'timestamp',
        'time': 'timestamp',
        'created_at': 'timestamp',
        'review_date': 'timestamp'
    }

    # Convert column names to lowercase for matching
    df_columns_lower = [col.lower() for col in df.columns]

    # Apply mappings
    new_columns = []
    for col in df.columns:
        col_lower = col.lower()
        if col_lower in column_mappings:
            new_columns.append(column_mappings[col_lower])
        else:
            new_columns.append(col)

    df.columns = new_columns

    # Ensure we have required columns
    required_columns = ['review_text', 'rating']
    for col in required_columns:
        if col not in df.columns:
            if col == 'review_text':
                # Try to find any text column
                text_cols = [c for c in df.columns if 'text' in c.lower() or 'review' in c.lower() or 'comment' in c.lower()]
                if text_cols:
                    df['review_text'] = df[text_cols[0]]
                else:
                    print(f"⚠️ Could not find text column, creating placeholder")
                    df['review_text'] = "Sample review text"
            elif col == 'rating':
                # Try to find any rating column
                rating_cols = [c for c in df.columns if 'rating' in c.lower() or 'star' in c.lower() or 'score' in c.lower()]
                if rating_cols:
                    df['rating'] = df[rating_cols[0]]
                else:
                    print(f"⚠️ Could not find rating column, creating placeholder")
                    df['rating'] = 3  # Default neutral rating

    # Add missing optional columns
    if 'business_name' not in df.columns:
        df['business_name'] = 'Unknown Business'
    if 'user_id' not in df.columns:
        df['user_id'] = [f'user_{i}' for i in range(len(df))]
    if 'timestamp' not in df.columns:
        df['timestamp'] = pd.date_range('2024-01-01', periods=len(df), freq='D')

    return df

In [17]:
# Load the dataset
df = load_dataset('./kaggle_data/reviews.csv')

# 👇 Simulate a bad row (make the 5th row's text missing)
df.loc[4, "review_text"] = ""   # or "" to test empty-string removal
print("🔧 Introduced a missing value in row 5 (text column)\n")

df = clean_reviews_dataset(df)

print("\n📋 Cleaned Dataset Info:")
print(df.info())
print(f"\n📊 Dataset shape: {df.shape}")
print("\n🔍 First 5 reviews:")
print(df.head())

# Display data quality info
print(f"\n✅ Data Quality Check:")
print(f"- Total reviews: {len(df)}")
print(f"- Unique businesses: {df['business_name'].nunique()}")
print(f"- Rating distribution: {dict(df['rating'].value_counts().sort_index())}")
print(f"- Missing values: {df.isnull().sum().sum()}")
print(f"- Average review length: {df['text'].str.len().mean():.1f} characters")

✅ Loaded 1100 rows from ./kaggle_data/reviews.csv
🔧 Introduced a missing value in row 5 (text column)

🧹 Cleaned dataset: 1100 → 1100 rows (removed 0)

📋 Cleaned Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1100 entries, 0 to 1099
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   business_name    1100 non-null   object
 1   author_name      1100 non-null   object
 2   text             1100 non-null   object
 3   rating           1100 non-null   int64 
 4   photo            1100 non-null   object
 5   rating_category  1100 non-null   object
dtypes: int64(1), object(5)
memory usage: 51.7+ KB
None

📊 Dataset shape: (1100, 6)

🔍 First 5 reviews:
                     business_name    author_name  \
0  Haci'nin Yeri - Yigit Lokantasi    Gulsum Akar   
1  Haci'nin Yeri - Yigit Lokantasi  Oguzhan Cetin   
2  Haci'nin Yeri - Yigit Lokantasi     Yasin Kuyu   
3  Haci'nin Yeri - Yigit Lokantasi     Orh

In [18]:
# Data Cleanup

def _find_col(df, aliases, required=True):
    """Return the first matching column from aliases; None if not found and required=False."""
    cols_lower = {c.lower(): c for c in df.columns}
    for a in aliases:
        if a.lower() in cols_lower:
            return cols_lower[a.lower()]
    if required:
        raise KeyError(f"None of the aliases {aliases} found in columns: {list(df.columns)}")
    return None

def clean_reviews_dataset(df):
    """
    Keep rows that have ALL of the following (non-empty, non-NaN):
      - business_name
      - author_name
      - text
      - rating
    Allow missing: photo, rating_category
    Preserve output columns in original schema.
    """

    # Resolve columns even if earlier steps renamed them
    col_business = _find_col(df, ["business_name", "restaurant", "place_name", "name"])
    col_author   = _find_col(df, ["author_name", "user", "user_name", "reviewer"])
    col_text     = _find_col(df, ["text", "review_text", "comment", "content"])
    col_rating   = _find_col(df, ["rating", "stars", "score", "star_rating"])

    # Optional columns may or may not exist
    col_photo          = _find_col(df, ["photo"], required=False)
    col_rating_category= _find_col(df, ["rating_category"], required=False)

    # Work on a copy
    d = df.copy()

    # Normalize whitespace for string fields (only if they exist)
    for c in [col_business, col_author, col_text]:
        d[c] = d[c].astype(str).str.strip()

    # Coerce rating to numeric
    d[col_rating] = pd.to_numeric(d[col_rating], errors="coerce")

    # Drop rows with missing/empty required fields
    before = len(d)
    d = d.dropna(subset=[col_business, col_author, col_text, col_rating])
    # Remove empty-string rows in required text columns
    for c in [col_business, col_author, col_text]:
        d = d[d[c] != ""]
    # Optionally enforce valid rating range (comment out if you want raw)
    d = d[(d[col_rating] >= 1) & (d[col_rating] <= 5)]

    removed = before - len(d)
    print(f"🧹 Cleaned dataset: {before} → {len(d)} rows (removed {removed})")

    # Rebuild output with your target column names in the same format
    out = pd.DataFrame({
        "business_name":    d[col_business],
        "author_name":      d[col_author],
        "text":             d[col_text],
        "rating":           d[col_rating],
    })

    # Attach optional columns if present; else create with NaN
    out["photo"] = d[col_photo] if col_photo in d.columns else pd.Series([pd.NA]*len(d))
    out["rating_category"] = d[col_rating_category] if col_rating_category in d.columns else pd.Series([pd.NA]*len(d))

    # Keep any extra columns? If you want to strictly keep only the six, return `out` as is.
    return out

## 🔧 Feature Engineering

Extract comprehensive textual and non-textual features from review data for ML models.

In [19]:
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
import string
from datetime import datetime

# Try to import textstat, use fallback if not available
try:
    from textstat import flesch_reading_ease, flesch_kincaid_grade
    TEXTSTAT_AVAILABLE = True
    print("✅ textstat module loaded successfully")
except ImportError:
    print("⚠️ textstat module not found. Readability metrics will be set to default values.")
    print("💡 To install: pip install textstat")
    TEXTSTAT_AVAILABLE = False
    
    # Fallback functions
    def flesch_reading_ease(text):
        return 50.0  # Default neutral readability score
    
    def flesch_kincaid_grade(text):
        return 8.0   # Default grade level

class AdvancedFeatureExtractor:
    """Extract comprehensive textual and non-textual features from review data"""
    
    def __init__(self):
        self.sia = SentimentIntensityAnalyzer()
        self.stop_words = set(stopwords.words('english'))
        
        # Common spam/promotional keywords
        self.spam_keywords = [
            'discount', 'promo', 'deal', 'offer', 'sale', 'buy', 'purchase', 
            'visit', 'click', 'link', 'website', 'free', 'win', 'prize'
        ]
        
        # Restaurant-related keywords for relevancy
        self.restaurant_keywords = [
            'food', 'meal', 'eat', 'taste', 'flavor', 'delicious', 'menu',
            'service', 'waiter', 'waitress', 'staff', 'cook', 'chef',
            'restaurant', 'cafe', 'dine', 'dining', 'lunch', 'dinner',
            'breakfast', 'appetizer', 'entree', 'dessert', 'drink'
        ]
        
    def extract_textual_features(self, text):
        """Extract comprehensive textual features"""
        features = {}
        text_lower = text.lower()
        words = text.split()
        
        # Basic text statistics
        features['text_length'] = len(text)
        features['word_count'] = len(words)
        features['sentence_count'] = len([s for s in text.split('.') if s.strip()])
        features['avg_word_length'] = np.mean([len(word) for word in words]) if words else 0
        
        # Character-level features
        features['uppercase_count'] = sum(1 for c in text if c.isupper())
        features['punctuation_count'] = sum(1 for c in text if c in string.punctuation)
        features['digit_count'] = sum(1 for c in text if c.isdigit())
        features['exclamation_count'] = text.count('!')
        features['question_count'] = text.count('?')
        
        # Ratios
        total_chars = len(text) if len(text) > 0 else 1
        features['uppercase_ratio'] = features['uppercase_count'] / total_chars
        features['punctuation_ratio'] = features['punctuation_count'] / total_chars
        features['digit_ratio'] = features['digit_count'] / total_chars
        
        # Sentiment analysis
        sentiment_scores = self.sia.polarity_scores(text)
        features.update({
            'sentiment_pos': sentiment_scores['pos'],
            'sentiment_neu': sentiment_scores['neu'],
            'sentiment_neg': sentiment_scores['neg'],
            'sentiment_compound': sentiment_scores['compound']
        })
        
        # URL and contact detection
        features['has_url'] = bool(re.search(r'http[s]?://\S+|www\.\w+\.\w+', text_lower))
        features['has_email'] = bool(re.search(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', text))
        features['has_phone'] = bool(re.search(r'\b\d{3}[-.]?\d{3}[-.]?\d{4}\b', text))
        
        # Personal pronouns and perspective
        first_person_words = ['i', 'me', 'my', 'myself', 'we', 'us', 'our']
        second_person_words = ['you', 'your', 'yours']
        third_person_words = ['he', 'she', 'it', 'they', 'them', 'their']
        
        words_lower = [w.lower().strip(string.punctuation) for w in words]
        features['first_person_count'] = sum(1 for word in words_lower if word in first_person_words)
        features['second_person_count'] = sum(1 for word in words_lower if word in second_person_words)
        features['third_person_count'] = sum(1 for word in words_lower if word in third_person_words)
        
        total_words = len(words) if len(words) > 0 else 1
        features['first_person_ratio'] = features['first_person_count'] / total_words
        features['second_person_ratio'] = features['second_person_count'] / total_words
        features['third_person_ratio'] = features['third_person_count'] / total_words
        
        # Spam/promotional indicators
        features['spam_keyword_count'] = sum(1 for keyword in self.spam_keywords if keyword in text_lower)
        features['spam_keyword_ratio'] = features['spam_keyword_count'] / total_words
        
        # Restaurant relevancy indicators
        features['restaurant_keyword_count'] = sum(1 for keyword in self.restaurant_keywords if keyword in text_lower)
        features['restaurant_keyword_ratio'] = features['restaurant_keyword_count'] / total_words
        
        # Readability metrics with fallback
        try:
            if TEXTSTAT_AVAILABLE:
                features['flesch_reading_ease'] = flesch_reading_ease(text)
                features['flesch_kincaid_grade'] = flesch_kincaid_grade(text)
            else:
                # Use simple fallback calculations
                avg_sentence_length = features['word_count'] / max(features['sentence_count'], 1)
                features['flesch_reading_ease'] = max(0, min(100, 206.835 - (1.015 * avg_sentence_length) - (84.6 * features['avg_word_length'])))
                features['flesch_kincaid_grade'] = max(0, (0.39 * avg_sentence_length) + (11.8 * features['avg_word_length']) - 15.59)
        except Exception as e:
            print(f"⚠️ Readability calculation failed: {e}")
            features['flesch_reading_ease'] = 50.0  # Default neutral score
            features['flesch_kincaid_grade'] = 8.0   # Default grade level
        
        # All caps words (often indicates spam/shouting)
        all_caps_words = [w for w in words if w.isupper() and len(w) > 1]
        features['all_caps_word_count'] = len(all_caps_words)
        features['all_caps_ratio'] = len(all_caps_words) / total_words
        
        # Repetitive patterns
        unique_words = set(words_lower)
        features['unique_word_ratio'] = len(unique_words) / total_words
        
        return features
    
    def extract_non_textual_features(self, row):
        """Extract non-textual features from metadata"""
        features = {}
        
        # Rating-based features
        features['rating'] = row['rating']
        features['is_extreme_rating'] = 1 if row['rating'] in [1, 5] else 0
        features['is_low_rating'] = 1 if row['rating'] <= 2 else 0
        features['is_high_rating'] = 1 if row['rating'] >= 4 else 0
        features['is_neutral_rating'] = 1 if row['rating'] == 3 else 0
        
        # Author-based features
        if 'author_name' in row:
            author_name = str(row['author_name'])
            features['author_name_length'] = len(author_name)
            features['author_has_numbers'] = 1 if any(c.isdigit() for c in author_name) else 0
            features['author_all_caps'] = 1 if author_name.isupper() else 0
        else:
            features['author_name_length'] = 0
            features['author_has_numbers'] = 0
            features['author_all_caps'] = 0
        
        # Business-based features
        if 'business_name' in row:
            business_name = str(row['business_name'])
            features['business_name_length'] = len(business_name)
        else:
            features['business_name_length'] = 0
        
        # Photo presence
        if 'photo' in row and pd.notna(row['photo']):
            features['has_photo'] = 1
        else:
            features['has_photo'] = 0
        
        return features
    
    def extract_all_features(self, df):
        """Extract all features for the entire dataset"""
        print("🔧 Extracting features...")
        if not TEXTSTAT_AVAILABLE:
            print("⚠️ Using fallback readability calculations (textstat not available)")
        
        # 🔍 COLUMN NAME DETECTION AND VALIDATION
        print(f"📋 Available columns: {list(df.columns)}")
        
        # Find the text column dynamically
        text_column = None
        possible_text_columns = ['text', 'review_text', 'review', 'content', 'comment']
        
        for col in possible_text_columns:
            if col in df.columns:
                text_column = col
                break
        
        if text_column is None:
            raise ValueError(f"❌ No text column found! Available columns: {list(df.columns)}")
        
        print(f"✅ Using '{text_column}' as the text column")
        
        all_features = []
        
        for idx, row in df.iterrows():
            if idx % 1000 == 0:
                print(f"   Processing row {idx}/{len(df)}")
            
            # Use the detected text column instead of hardcoded 'review_text'
            text_features = self.extract_textual_features(row[text_column])
            non_text_features = self.extract_non_textual_features(row)
            
            # Combine all features
            combined_features = {**text_features, **non_text_features}
            all_features.append(combined_features)
        
        features_df = pd.DataFrame(all_features)
        print(f"✅ Feature extraction complete! Extracted {len(features_df.columns)} features")
        return features_df

# Initialize feature extractor
feature_extractor = AdvancedFeatureExtractor()

# Extract features from the dataset
features_df = feature_extractor.extract_all_features(df)

# Combine with original data
df_with_features = pd.concat([df.reset_index(drop=True), features_df], axis=1)

print(f"\n📊 Dataset with features shape: {df_with_features.shape}")
print(f"\n🔍 Feature columns extracted:")
for i, col in enumerate(features_df.columns, 1):
    print(f"   {i:2d}. {col}")

# Display feature statistics
print("\n📈 Feature Statistics Summary:")
print(features_df.describe().round(3))

✅ textstat module loaded successfully
🔧 Extracting features...
📋 Available columns: ['business_name', 'author_name', 'text', 'rating', 'photo', 'rating_category']
✅ Using 'text' as the text column
   Processing row 0/1100
   Processing row 1000/1100
✅ Feature extraction complete! Extracted 44 features

📊 Dataset with features shape: (1100, 50)

🔍 Feature columns extracted:
    1. text_length
    2. word_count
    3. sentence_count
    4. avg_word_length
    5. uppercase_count
    6. punctuation_count
    7. digit_count
    8. exclamation_count
    9. question_count
   10. uppercase_ratio
   11. punctuation_ratio
   12. digit_ratio
   13. sentiment_pos
   14. sentiment_neu
   15. sentiment_neg
   16. sentiment_compound
   17. has_url
   18. has_email
   19. has_phone
   20. first_person_count
   21. second_person_count
   22. third_person_count
   23. first_person_ratio
   24. second_person_ratio
   25. third_person_ratio
   26. spam_keyword_count
   27. spam_keyword_ratio
   28. restau

## 🚫 Policy Detection Module

Implement rule-based and ML-based policy violation detectors for the three main categories.

In [20]:
import re
from typing import Dict, Tuple, List
import warnings
warnings.filterwarnings('ignore')

class PolicyViolationDetector:
    """Rule-based policy violation detector for restaurant reviews"""
    
    def __init__(self):
        # Advertisement detection patterns
        self.ad_patterns = [
            r'\b(?:call|text|contact|visit|website|phone|email|dm|message)\s+(?:us|me|now|today)\b',
            r'\b(?:best|cheapest|lowest|highest|top)\s+(?:price|deal|offer|service)\b',
            r'\b(?:free|discount|sale|promo|special|offer|deal)\b.*\b(?:today|now|limited|expires)\b',
            r'\b(?:check|visit|see|follow)\s+(?:our|my)\s+(?:website|page|profile|instagram|facebook)\b',
            r'\b(?:book|order|reserve)\s+(?:now|today|online)\b',
            r'(?:www\.|http|\.com|\.org|\.net)',
            r'\b(?:delivery|takeout|pickup)\s+(?:available|service)\b',
            r'\b(?:new|grand)\s+opening\b',
            r'\b(?:hiring|recruiting|looking\s+for)\b'
        ]
        
        # Irrelevant content patterns
        self.irrelevant_patterns = [
            r'\b(?:politics|election|government|president|mayor|council)\b',
            r'\b(?:religion|church|mosque|temple|spiritual)\b',
            r'\b(?:personal|relationship|dating|marriage|divorce)\b',
            r'\b(?:medical|health|doctor|hospital|surgery|medicine)\b',
            r'\b(?:school|education|homework|exam|grade)\b',
            r'\b(?:weather|rain|snow|sunny|cloudy)\b',
            r'\b(?:sports|game|match|team|player|score)\b',
            r'\b(?:movie|film|tv|show|actor|actress)\b',
            r'\b(?:music|song|concert|band|album)\b',
            r'\b(?:car|vehicle|traffic|parking|driving)\b'
        ]
        
        # Rant without visit patterns
        self.rant_patterns = [
            r'\b(?:never\s+(?:been|visited|went)|haven\'t\s+(?:been|visited))\b',
            r'\b(?:heard|read|saw)\s+(?:about|reviews|complaints)\b',
            r'\b(?:based\s+on|according\s+to)\s+(?:reviews|others|friends)\b',
            r'\b(?:planning\s+to|might|considering)\s+(?:visit|go|try)\b',
            r'\b(?:looks|seems|appears)\s+(?:bad|terrible|awful|horrible)\b',
            r'\b(?:reputation|known\s+for)\s+(?:being|having)\b',
            r'\b(?:everyone\s+says|people\s+say|i\'ve\s+heard)\b'
        ]
        
        # Restaurant-related keywords (for relevance check)
        self.restaurant_keywords = [
            'food', 'meal', 'dish', 'restaurant', 'cafe', 'bar', 'service', 'waiter', 'waitress',
            'menu', 'order', 'taste', 'flavor', 'delicious', 'cook', 'chef', 'kitchen',
            'eat', 'dine', 'dining', 'lunch', 'dinner', 'breakfast', 'appetizer', 'dessert',
            'drink', 'beverage', 'wine', 'beer', 'cocktail', 'table', 'reservation', 'staff'
        ]
    
    def detect_advertisement(self, text: str) -> Tuple[bool, float, List[str]]:
        """Detect advertisement content"""
        text_lower = text.lower()
        matches = []
        score = 0
        
        for pattern in self.ad_patterns:
            if re.search(pattern, text_lower):
                matches.append(pattern)
                score += 1
        
        # Normalize score
        confidence = min(score / len(self.ad_patterns), 1.0)
        is_ad = confidence > 0.3
        
        return is_ad, confidence, matches
    
    def detect_irrelevant_content(self, text: str) -> Tuple[bool, float, List[str]]:
        """Detect irrelevant content"""
        text_lower = text.lower()
        matches = []
        irrelevant_score = 0
        relevant_score = 0
        
        # Check for irrelevant patterns
        for pattern in self.irrelevant_patterns:
            if re.search(pattern, text_lower):
                matches.append(pattern)
                irrelevant_score += 1
        
        # Check for restaurant relevance
        for keyword in self.restaurant_keywords:
            if keyword in text_lower:
                relevant_score += 1
        
        # Calculate confidence
        total_patterns = len(self.irrelevant_patterns)
        if irrelevant_score > 0 and relevant_score == 0:
            confidence = min(irrelevant_score / total_patterns, 1.0)
            is_irrelevant = confidence > 0.2
        else:
            confidence = max(0, (irrelevant_score - relevant_score * 0.5) / total_patterns)
            is_irrelevant = confidence > 0.3
        
        return is_irrelevant, max(confidence, 0), matches
    
    def detect_rant_without_visit(self, text: str) -> Tuple[bool, float, List[str]]:
        """Detect rants without actual visit"""
        text_lower = text.lower()
        matches = []
        score = 0
        
        for pattern in self.rant_patterns:
            if re.search(pattern, text_lower):
                matches.append(pattern)
                score += 1
        
        # Additional checks for negative sentiment without visit indicators
        negative_words = ['terrible', 'awful', 'horrible', 'worst', 'disgusting', 'hate']
        visit_indicators = ['went', 'visited', 'ate', 'ordered', 'tried', 'had dinner', 'had lunch']
        
        has_negative = any(word in text_lower for word in negative_words)
        has_visit = any(indicator in text_lower for indicator in visit_indicators)
        
        if has_negative and not has_visit:
            score += 0.5
        
        # Normalize score
        confidence = min(score / len(self.rant_patterns), 1.0)
        is_rant = confidence > 0.3
        
        return is_rant, confidence, matches
    
    def analyze_review(self, text: str) -> Dict:
        """Comprehensive policy violation analysis"""
        if not text or len(text.strip()) < 10:
            return {
                'is_violation': False,
                'violation_type': None,
                'confidence': 0.0,
                'details': 'Text too short for analysis'
            }
        
        # Run all detectors
        is_ad, ad_conf, ad_matches = self.detect_advertisement(text)
        is_irrelevant, irr_conf, irr_matches = self.detect_irrelevant_content(text)
        is_rant, rant_conf, rant_matches = self.detect_rant_without_visit(text)
        
        # Determine primary violation
        violations = [
            ('advertisement', ad_conf, ad_matches),
            ('irrelevant_content', irr_conf, irr_matches),
            ('rant_without_visit', rant_conf, rant_matches)
        ]
        
        violations.sort(key=lambda x: x[1], reverse=True)
        primary_violation = violations[0]
        
        is_violation = primary_violation[1] > 0.3
        
        return {
            'is_violation': is_violation,
            'violation_type': primary_violation[0] if is_violation else None,
            'confidence': primary_violation[1],
            'all_scores': {
                'advertisement': ad_conf,
                'irrelevant_content': irr_conf,
                'rant_without_visit': rant_conf
            },
            'matches': primary_violation[2] if is_violation else [],
            'details': f"Primary violation: {primary_violation[0]}" if is_violation else "No violation detected"
        }

# Initialize policy detector
policy_detector = PolicyViolationDetector()

# Test with sample reviews
test_reviews = [
    "Great food and excellent service! The pasta was amazing.",
    "Call us now for the best deals! Visit our website www.example.com",
    "I hate politics and this election is terrible. Nothing about food here.",
    "I heard this place is awful, never been there but people say it's bad."
]

print("🔍 Policy Violation Detection Results:")
print("=" * 50)

for i, review in enumerate(test_reviews, 1):
    result = policy_detector.analyze_review(review)
    print(f"\n📝 Review {i}: '{review[:50]}...'")
    print(f"🚨 Violation: {result['is_violation']}")
    if result['is_violation']:
        print(f"📋 Type: {result['violation_type']}")
        print(f"🎯 Confidence: {result['confidence']:.3f}")
        print(f"📊 All Scores: {result['all_scores']}")
    print(f"💡 Details: {result['details']}")

🔍 Policy Violation Detection Results:

📝 Review 1: 'Great food and excellent service! The pasta was am...'
🚨 Violation: False
💡 Details: No violation detected

📝 Review 2: 'Call us now for the best deals! Visit our website ...'
🚨 Violation: True
📋 Type: advertisement
🎯 Confidence: 0.333
📊 All Scores: {'advertisement': 0.3333333333333333, 'irrelevant_content': 0, 'rant_without_visit': 0.0}
💡 Details: Primary violation: advertisement

📝 Review 3: 'I hate politics and this election is terrible. Not...'
🚨 Violation: False
💡 Details: No violation detected

📝 Review 4: 'I heard this place is awful, never been there but ...'
🚨 Violation: True
📋 Type: rant_without_visit
🎯 Confidence: 0.357
📊 All Scores: {'advertisement': 0.0, 'irrelevant_content': 0, 'rant_without_visit': 0.35714285714285715}
💡 Details: Primary violation: rant_without_visit


## 🤖 Gemma 3 12B Model Integration

Using Google's Gemma 3 12B model with HuggingFace Inference Client for advanced policy detection and review classification.

In [21]:
from huggingface_hub import InferenceClient
import json
import time
from typing import Dict, List, Optional
import os

class GemmaReviewClassifier:
    """Advanced review classifier using Gemma 3 12B model"""
    
    def __init__(self, hf_token: Optional[str] = None):
        """
        Initialize Gemma classifier
        
        Args:
            hf_token: HuggingFace token (optional, can be set in environment)
        """
        self.model_name = "google/gemma-3-12b-it"
        
        # Set up HuggingFace token
        if hf_token:
            os.environ["HUGGINGFACE_HUB_TOKEN"] = hf_token
        
        try:
            self.client = InferenceClient(
                model=self.model_name,
                token=hf_token or os.getenv("HUGGINGFACE_HUB_TOKEN")
            )
            print(f"✅ Successfully initialized Gemma 3 12B model: {self.model_name}")
        except Exception as e:
            print(f"⚠️ Warning: Could not initialize model. Error: {e}")
            print("💡 Note: You may need to provide a HuggingFace token or use a fallback model")
            self.client = None
    
    def create_policy_prompt(self, review_text: str) -> str:
        """Create a structured prompt for policy violation detection"""
        
        prompt = f"""You are an expert content moderator for restaurant review platforms. Analyze the following review and determine if it violates any of these policies:

1. **Advertisement**: Reviews that promote businesses, include contact information, or solicit customers
2. **Irrelevant Content**: Reviews about topics unrelated to the restaurant experience
3. **Rant Without Visit**: Negative reviews from people who haven't actually visited the restaurant

Review to analyze: "{review_text}"

Please respond with a JSON object in this exact format:
{{
    "is_violation": true/false,
    "violation_type": "advertisement" or "irrelevant_content" or "rant_without_visit" or null,
    "confidence": 0.0-1.0,
    "reasoning": "Brief explanation of your decision",
    "is_trustworthy": true/false,
    "sentiment": "positive" or "negative" or "neutral"
}}

Focus on:
- Clear policy violations vs. legitimate reviews
- Evidence of actual restaurant visit
- Commercial intent vs. genuine feedback
- Restaurant relevance vs. off-topic content

Response:"""
        
        return prompt
    
    def create_quality_prompt(self, review_text: str) -> str:
        """Create a prompt for review quality assessment"""
        
        prompt = f"""As an expert in restaurant review quality assessment, evaluate this review for trustworthiness and usefulness:

Review: "{review_text}"

Assess the review on these dimensions:
1. **Authenticity**: Does this seem like a genuine customer experience?
2. **Specificity**: Does it provide specific details about food, service, or atmosphere?
3. **Helpfulness**: Would this review help other customers make decisions?
4. **Balance**: Does it provide constructive feedback rather than just complaints?

Respond with JSON:
{{
    "quality_score": 0.0-1.0,
    "authenticity": 0.0-1.0,
    "specificity": 0.0-1.0,
    "helpfulness": 0.0-1.0,
    "is_spam": true/false,
    "is_fake": true/false,
    "key_insights": ["insight1", "insight2"],
    "recommendation": "keep" or "flag" or "remove"
}}

Response:"""
        
        return prompt
    
    def classify_review(self, review_text: str, max_retries: int = 3) -> Dict:
        """Classify a review for policy violations and quality"""
        
        if not self.client:
            return {
                "error": "Model not available",
                "fallback": "Using rule-based detection only"
            }
        
        if not review_text or len(review_text.strip()) < 5:
            return {
                "error": "Review text too short",
                "is_violation": False,
                "quality_score": 0.0
            }
        
        results = {}
        
        # Policy violation detection
        try:
            policy_prompt = self.create_policy_prompt(review_text)
            
            for attempt in range(max_retries):
                try:
                    policy_response = self.client.text_generation(
                        policy_prompt,
                        max_new_tokens=200,
                        temperature=0.1,
                        do_sample=True,
                        return_full_text=False
                    )
                    
                    # Parse JSON response
                    policy_data = self._parse_json_response(policy_response)
                    if policy_data:
                        results['policy'] = policy_data
                        break
                        
                except Exception as e:
                    if attempt == max_retries - 1:
                        results['policy_error'] = str(e)
                    time.sleep(1)
        
        except Exception as e:
            results['policy_error'] = str(e)
        
        # Quality assessment
        try:
            quality_prompt = self.create_quality_prompt(review_text)
            
            for attempt in range(max_retries):
                try:
                    quality_response = self.client.text_generation(
                        quality_prompt,
                        max_new_tokens=200,
                        temperature=0.1,
                        do_sample=True,
                        return_full_text=False
                    )
                    
                    # Parse JSON response
                    quality_data = self._parse_json_response(quality_response)
                    if quality_data:
                        results['quality'] = quality_data
                        break
                        
                except Exception as e:
                    if attempt == max_retries - 1:
                        results['quality_error'] = str(e)
                    time.sleep(1)
        
        except Exception as e:
            results['quality_error'] = str(e)
        
        return self._consolidate_results(results)
    
    def _parse_json_response(self, response: str) -> Optional[Dict]:
        """Parse JSON from model response"""
        try:
            # Find JSON in response
            start_idx = response.find('{')
            end_idx = response.rfind('}') + 1
            
            if start_idx != -1 and end_idx != -1:
                json_str = response[start_idx:end_idx]
                return json.loads(json_str)
            
        except (json.JSONDecodeError, ValueError) as e:
            print(f"JSON parsing error: {e}")
            print(f"Response: {response[:200]}...")
        
        return None
    
    def _consolidate_results(self, results: Dict) -> Dict:
        """Consolidate policy and quality results"""
        
        consolidated = {
            'timestamp': time.time(),
            'model_used': self.model_name
        }
        
        # Policy results
        if 'policy' in results:
            policy = results['policy']
            consolidated.update({
                'is_violation': policy.get('is_violation', False),
                'violation_type': policy.get('violation_type'),
                'policy_confidence': policy.get('confidence', 0.0),
                'is_trustworthy': policy.get('is_trustworthy', True),
                'sentiment': policy.get('sentiment', 'neutral'),
                'policy_reasoning': policy.get('reasoning', '')
            })
        else:
            consolidated.update({
                'is_violation': False,
                'violation_type': None,
                'policy_confidence': 0.0,
                'policy_error': results.get('policy_error', 'Unknown error')
            })
        
        # Quality results
        if 'quality' in results:
            quality = results['quality']
            consolidated.update({
                'quality_score': quality.get('quality_score', 0.5),
                'authenticity': quality.get('authenticity', 0.5),
                'specificity': quality.get('specificity', 0.5),
                'helpfulness': quality.get('helpfulness', 0.5),
                'is_spam': quality.get('is_spam', False),
                'is_fake': quality.get('is_fake', False),
                'key_insights': quality.get('key_insights', []),
                'recommendation': quality.get('recommendation', 'keep')
            })
        else:
            consolidated.update({
                'quality_score': 0.5,
                'quality_error': results.get('quality_error', 'Unknown error')
            })
        
        return consolidated
    
    def batch_classify(self, reviews: List[str], batch_size: int = 5) -> List[Dict]:
        """Classify multiple reviews in batches"""
        
        results = []
        
        for i in range(0, len(reviews), batch_size):
            batch = reviews[i:i + batch_size]
            print(f"🔄 Processing batch {i//batch_size + 1}/{(len(reviews)-1)//batch_size + 1}")
            
            batch_results = []
            for review in batch:
                result = self.classify_review(review)
                batch_results.append(result)
                time.sleep(0.5)  # Rate limiting
            
            results.extend(batch_results)
        
        return results

# Initialize Gemma classifier
print("🚀 Initializing Gemma 3 12B Classifier...")
print("💡 Note: You may need a HuggingFace token for full functionality")

# For Colab users, uncomment and add your token:
# gemma_classifier = GemmaReviewClassifier(hf_token="your_hf_token_here")

# For token-less testing:
try:
    gemma_classifier = GemmaReviewClassifier()
except Exception as e:
    print(f"⚠️ Could not initialize Gemma model: {e}")
    print("🔄 Continuing with rule-based detection only...")
    gemma_classifier = None

🚀 Initializing Gemma 3 12B Classifier...
💡 Note: You may need a HuggingFace token for full functionality
✅ Successfully initialized Gemma 3 12B model: google/gemma-3-12b-it


In [22]:
# 🚀 Sample Review Classification - 200 Reviews with Text and Labels
print("=" * 70)
print("🤖 SAMPLE REVIEW CLASSIFICATION - 200 REVIEWS")
print("=" * 70)

def load_additional_reviews():
    """Load additional review datasets - simplified to extract only text and rating"""
    additional_dfs = []
    
    # Check for additional review files
    additional_files = [
        './review-other.json',
        # Add more file paths as needed
    ]
    1
    for file_path in additional_files:
        if os.path.exists(file_path):
            try:
                if file_path.endswith('.json'):
                    # Enhanced JSON loading with better error handling
                    print(f"🔄 Attempting to load {file_path}...")
                    
                    # First, peek at the file to determine format
                    with open(file_path, 'r', encoding='utf-8') as f:
                        first_line = f.readline().strip()
                        second_line = f.readline().strip()
                    
                    # Check if it's JSONL format (each line is a JSON object)
                    if (first_line.startswith('{') and first_line.endswith('}') and 
                        second_line.startswith('{') and second_line.endswith('}')):
                        
                        print(f"📄 Detected JSONL format (JSON Lines) - parsing line by line...")
                        json_objects = []
                        
                        with open(file_path, 'r', encoding='utf-8') as f:
                            for line_num, line in enumerate(f, 1):
                                line = line.strip()
                                if line:  # Skip empty lines
                                    try:
                                        obj = json.loads(line)
                                        json_objects.append(obj)
                                    except json.JSONDecodeError as e:
                                        print(f"⚠️ Skipping invalid JSON on line {line_num}: {e}")
                                        continue
                        
                        json_data = json_objects
                        print(f"✅ Successfully parsed {len(json_data)} JSON objects from JSONL file")
                        
                    else:
                        # Try standard JSON formats
                        with open(file_path, 'r', encoding='utf-8') as f:
                            content = f.read().strip()
                        
                        if content.startswith('[') and content.endswith(']'):
                            # Array of JSON objects
                            json_data = json.loads(content)
                            print(f"✅ Successfully parsed JSON array with {len(json_data)} objects")
                        elif content.startswith('{') and content.endswith('}'):
                            # Single JSON object
                            json_data = json.loads(content)
                            print(f"✅ Successfully parsed single JSON object")
                        else:
                            print(f"❌ Unrecognized JSON format in {file_path}")
                            continue
                    
                    # Convert to DataFrame and apply same cleaning as Kaggle data
                    if isinstance(json_data, list):
                        if json_data:  # Check if list is not empty
                            full_df = pd.DataFrame(json_data)
                            print(f"✅ Created DataFrame with {len(full_df)} rows and {len(full_df.columns)} columns")
                            print(f"📋 Original columns: {list(full_df.columns)}")
                            
                            # Apply same cleaning process as Kaggle dataset
                            cleaned_df = clean_json_data_like_kaggle(full_df)
                            if cleaned_df is not None and len(cleaned_df) > 0:
                                additional_dfs.append(cleaned_df)
                        else:
                            print(f"⚠️ JSON file {file_path} contains empty array")
                    else:
                        full_df = pd.json_normalize(json_data)
                        print(f"✅ Created DataFrame with {len(full_df)} rows and {len(full_df.columns)} columns")
                        cleaned_df = clean_json_data_like_kaggle(full_df)
                        if cleaned_df is not None and len(cleaned_df) > 0:
                            additional_dfs.append(cleaned_df)
                        
                elif file_path.endswith('.html'):
                    # Enhanced HTML parsing with dependency installation
                    try:
                        # First try to install lxml if not available
                        try:
                            import lxml
                        except ImportError:
                            print(f"📦 Installing lxml for HTML parsing...")
                            import subprocess
                            import sys
                            subprocess.check_call([sys.executable, "-m", "pip", "install", "lxml"])
                            print(f"✅ lxml installed successfully")
                        
                        # Now try to parse HTML
                        html_tables = pd.read_html(file_path, encoding='utf-8')
                        if html_tables:
                            # If multiple tables, try to find the one with review-like data
                            best_table = None
                            max_rows = 0
                            
                            for i, table in enumerate(html_tables):
                                if len(table) > max_rows:
                                    # Look for text-like columns
                                    text_like_cols = [col for col in table.columns if 
                                                    any(keyword in str(col).lower() for keyword in 
                                                        ['text', 'review', 'comment', 'content', 'message'])]
                                    if text_like_cols or len(table.columns) >= 3:  # Reasonable number of columns
                                        best_table = table
                                        max_rows = len(table)
                            
                            if best_table is not None:
                                print(f"✅ Found table with {len(best_table)} rows from {file_path}")
                                cleaned_df = clean_json_data_like_kaggle(best_table)
                                if cleaned_df is not None and len(cleaned_df) > 0:
                                    additional_dfs.append(cleaned_df)
                            else:
                                print(f"⚠️ No suitable table found in {file_path}")
                        else:
                            print(f"⚠️ No tables found in HTML file {file_path}")
                            
                    except Exception as e:
                        print(f"❌ Could not parse HTML file {file_path}: {e}")
                        print(f"💡 Try converting the HTML file to CSV format manually")
                        
                elif file_path.endswith('.csv'):
                    try:
                        # Try different encodings
                        encodings = ['utf-8', 'latin-1', 'cp1252', 'iso-8859-1']
                        full_df = None
                        
                        for encoding in encodings:
                            try:
                                full_df = pd.read_csv(file_path, encoding=encoding)
                                print(f"✅ Loaded {len(full_df)} rows from {file_path} (encoding: {encoding})")
                                break
                            except UnicodeDecodeError:
                                continue
                        
                        if full_df is not None:
                            cleaned_df = clean_json_data_like_kaggle(full_df)
                            if cleaned_df is not None and len(cleaned_df) > 0:
                                additional_dfs.append(cleaned_df)
                        else:
                            print(f"❌ Could not read CSV file {file_path} with any encoding")
                            
                    except Exception as e:
                        print(f"❌ Error loading CSV {file_path}: {e}")
                        
            except Exception as e:
                print(f"❌ Unexpected error loading {file_path}: {e}")
                import traceback
                print(f"📋 Full error traceback:")
                traceback.print_exc()
        else:
            print(f"ℹ️ File not found: {file_path}")
    
    return additional_dfs

def clean_json_data_like_kaggle(df):
    """Clean JSON data using the same method as Kaggle dataset (clean_reviews_dataset)"""
    
    print(f"🧹 Applying Kaggle-style cleaning to DataFrame with columns: {list(df.columns)}")
    
    try:
        # Use the same _find_col function and cleaning logic as Kaggle dataset
        col_business = _find_col(df, ["business_name", "restaurant", "place_name", "name"], required=False)
        col_author   = _find_col(df, ["author_name", "user", "user_name", "reviewer"], required=False)
        col_text     = _find_col(df, ["text", "review_text", "comment", "content", "review"], required=True)
        col_rating   = _find_col(df, ["rating", "stars", "score", "star_rating"], required=False)

        # Optional columns may or may not exist
        col_photo          = _find_col(df, ["photo"], required=False)
        col_rating_category= _find_col(df, ["rating_category"], required=False)

        # Work on a copy
        d = df.copy()

        # Normalize whitespace for string fields (only if they exist)
        if col_text:
            d[col_text] = d[col_text].astype(str).str.strip()
        if col_business:
            d[col_business] = d[col_business].astype(str).str.strip()
        if col_author:
            d[col_author] = d[col_author].astype(str).str.strip()

        # Coerce rating to numeric if exists
        if col_rating:
            d[col_rating] = pd.to_numeric(d[col_rating], errors="coerce")
            d[col_rating] = d[col_rating].fillna(3)  # Default to 3 if NaN
            d[col_rating] = d[col_rating].clip(1, 5)  # Ensure 1-5 range

        # Drop rows with missing/empty required fields
        before = len(d)
        d = d.dropna(subset=[col_text])
        # Remove empty-string rows in text column
        d = d[d[col_text] != ""]
        # Remove very short reviews
        d = d[d[col_text].str.len() >= 5]

        removed = before - len(d)
        print(f"🧹 Cleaned dataset: {before} → {len(d)} rows (removed {removed})")

        if len(d) == 0:
            print("⚠️ No valid reviews remaining after cleaning")
            return None

        # Rebuild output with target column names in the same format as Kaggle
        out_data = {
            "text": d[col_text],
            "rating": d[col_rating] if col_rating else pd.Series([3]*len(d))
        }
        
        # Add optional columns if they exist
        if col_business:
            out_data["business_name"] = d[col_business]
        else:
            out_data["business_name"] = pd.Series(['Unknown Business']*len(d))
            
        if col_author:
            out_data["author_name"] = d[col_author]
        else:
            out_data["author_name"] = pd.Series([f'user_{i}' for i in range(len(d))])

        out = pd.DataFrame(out_data)
        
        # Attach optional columns if present
        if col_photo and col_photo in d.columns:
            out["photo"] = d[col_photo]
        else:
            out["photo"] = pd.Series([pd.NA]*len(d))
            
        if col_rating_category and col_rating_category in d.columns:
            out["rating_category"] = d[col_rating_category]
        else:
            out["rating_category"] = pd.Series([pd.NA]*len(d))

        return out.reset_index(drop=True)
        
    except Exception as e:
        print(f"❌ Error during cleaning: {e}")
        return None

def merge_all_datasets_and_sample(kaggle_df, additional_dfs, sample_size=200):
    """Merge all datasets and sample specified number of reviews"""
    
    if not additional_dfs:
        print("ℹ️ No additional datasets found, using Kaggle data only")
        combined_df = kaggle_df.copy()
    else:
        print(f"🔄 Merging {len(additional_dfs)} additional datasets with Kaggle data...")
        
        # Combine all datasets
        all_dfs = [kaggle_df]
        
        for i, add_df in enumerate(additional_dfs):
            print(f"   Adding dataset {i+1}/{len(additional_dfs)} with {len(add_df)} rows")
            all_dfs.append(add_df)
        
        try:
            combined_df = pd.concat(all_dfs, ignore_index=True)
            
            # Remove duplicates based on text content
            original_len = len(combined_df)
            combined_df = combined_df.drop_duplicates(subset=['text'], keep='first')
            duplicates_removed = original_len - len(combined_df)
            
            print(f"📊 Merged dataset statistics:")
            print(f"   Total reviews: {len(combined_df)}")
            print(f"   Kaggle reviews: {len(kaggle_df)}")
            print(f"   Additional reviews: {len(combined_df) - len(kaggle_df)}")
            print(f"   Duplicates removed: {duplicates_removed}")
            
        except Exception as e:
            print(f"❌ Error merging datasets: {e}")
            print("🔄 Falling back to Kaggle data only")
            combined_df = kaggle_df.copy()
    
    # Sample the specified number of reviews
    if len(combined_df) > sample_size:
        print(f"🎯 Sampling {sample_size} reviews from {len(combined_df)} total reviews...")
        sampled_df = combined_df.sample(n=sample_size, random_state=42).reset_index(drop=True)
    else:
        print(f"🔄 Using all {len(combined_df)} reviews (less than requested {sample_size})")
        sampled_df = combined_df.reset_index(drop=True)
    
    print(f"📊 Final sample size: {len(sampled_df)} reviews")
    print(f"📋 Final columns: {list(sampled_df.columns)}")
    
    return sampled_df

# Test the JSONL parsing specifically
print("🧪 Testing JSONL parsing on your review-other.json file...")
test_path = './review-other.json'

if os.path.exists(test_path):
    try:
        # Load and show sample data
        sample_objects = []
        with open(test_path, 'r', encoding='utf-8') as f:
            for i, line in enumerate(f):
                if i >= 5:  # Just get first 5 lines for testing
                    break
                line = line.strip()
                if line:
                    try:
                        obj = json.loads(line)
                        sample_objects.append(obj)
                    except json.JSONDecodeError as e:
                        print(f"❌ Error parsing line {i+1}: {e}")
        
        if sample_objects:
            sample_df = pd.DataFrame(sample_objects)
            print(f"✅ Successfully parsed sample data:")
            print(f"📊 Shape: {sample_df.shape}")
            print(f"📋 Original columns: {list(sample_df.columns)}")
            
            # Test cleaning
            cleaned_sample = clean_json_data_like_kaggle(sample_df)
            if cleaned_sample is not None:
                print(f"✅ Cleaned sample:")
                print(f"📊 Shape: {cleaned_sample.shape}")
                print(f"📋 Columns: {list(cleaned_sample.columns)}")
                print(f"🔍 First few rows:")
                print(cleaned_sample.head(3))
        else:
            print("❌ No valid JSON objects found in sample")
            
    except Exception as e:
        print(f"❌ Error testing JSONL parsing: {e}")
else:
    print(f"❌ File not found: {test_path}")

# Check if we have a loaded dataframe
if 'df' in locals() and not df.empty:
    
    # Load additional review datasets
    print("📂 Searching for additional review datasets...")
    additional_datasets = load_additional_reviews()
    
    # Merge all datasets and sample 200 reviews
    sample_df = merge_all_datasets_and_sample(df, additional_datasets, sample_size=200)
    
    # Verify we have the required columns
    if 'text' not in sample_df.columns:
        print("❌ No text column found in sample dataset")
        print("🔄 Please check your data sources")
    else:
        print(f"📝 Using 'text' column for review text")
        
        # Display sample dataset information
        print(f"\n📊 SAMPLE DATASET INFORMATION:")
        print(f"   Total reviews for processing: {len(sample_df)}")
        print(f"   Columns: {list(sample_df.columns)}")
        
        if 'rating' in sample_df.columns:
            rating_dist = dict(sample_df['rating'].value_counts().sort_index())
            print(f"   Rating distribution: {rating_dist}")
        
        # Show a sample of the data
        print(f"\n🔍 Sample of data:")
        for i, row in sample_df.head(5).iterrows():
            text = str(row['text'])[:100]
            rating = row.get('rating', 'N/A')
            print(f"   {i+1}. Rating: {rating} | Text: {text}{'...' if len(str(row['text'])) > 100 else ''}")
        print()
        
        # Process reviews and generate labels
        if gemma_classifier and gemma_classifier.client:
            print(f"🔄 Processing {len(sample_df)} reviews with Gemma classifier...")
            print(f"🚨 RESULTS (Text + Labels):")
            print("-" * 60)
            
            results_for_output = []
            
            for idx, row in sample_df.iterrows():
                review_text = str(row['text']) if pd.notna(row['text']) else ""
                review_rating = row.get('rating', 3)
                
                # Progress indicator
                if (idx + 1) % 10 == 0:
                    print(f"\n📊 Progress: {idx + 1}/{len(sample_df)} reviews")
                    print("-" * 60)
                
                # Skip empty reviews
                if len(review_text.strip()) < 5:
                    results_for_output.append({
                        'index': idx + 1,
                        'text': review_text,
                        'rating': review_rating,
                        'label': 'INVALID',
                        'reason': 'Review too short'
                    })
                    continue
                
                try:
                    # Classify with Gemma
                    result = gemma_classifier.classify_review(review_text)
                    
                    # Determine label
                    is_violation = result.get('is_violation', False)
                    violation_type = result.get('violation_type', 'unknown')
                    confidence = result.get('policy_confidence', 0.0)
                    
                    if is_violation:
                        label = f"VIOLATION ({violation_type.upper()})"
                    else:
                        label = "LEGITIMATE"
                    
                    results_for_output.append({
                        'index': idx + 1,
                        'text': review_text,
                        'rating': review_rating,
                        'label': label,
                        'confidence': confidence,
                        'reason': result.get('policy_reasoning', '')
                    })
                    
                    # Display result
                    print(f"\n📝 Review #{idx + 1}: {label}")
                    print(f"⭐ Rating: {review_rating}")
                    print(f"🎯 Confidence: {confidence:.3f}")
                    print(f"📄 Text: \"{review_text[:150]}{'...' if len(review_text) > 150 else ''}\"")
                    if result.get('policy_reasoning'):
                        print(f"💡 Reason: {result.get('policy_reasoning', '')[:100]}{'...' if len(result.get('policy_reasoning', '')) > 100 else ''}")
                    
                except Exception as e:
                    results_for_output.append({
                        'index': idx + 1,
                        'text': review_text,
                        'rating': review_rating,
                        'label': 'ERROR',
                        'reason': str(e)
                    })
                    print(f"\n❌ ERROR processing review #{idx + 1}: {str(e)}")
                
                # Rate limiting
                time.sleep(0.8)
            
            # Final summary and save results
            print(f"\n" + "="*70)
            print(f"📊 PROCESSING COMPLETE!")
            
            # Count labels
            label_counts = {}
            for result in results_for_output:
                label = result['label'].split(' (')[0]  # Get main label part
                label_counts[label] = label_counts.get(label, 0) + 1
            
            print(f"📋 Label distribution:")
            for label, count in label_counts.items():
                print(f"   {label}: {count}")
            
            # Save results to CSV
            results_df = pd.DataFrame(results_for_output)
            output_filename = f"sample_200_reviews_with_labels_{int(time.time())}.csv"
            results_df.to_csv(output_filename, index=False)
            print(f"💾 Results saved to: {output_filename}")
            print(f"📊 Output columns: {list(results_df.columns)}")
            
            # Show first few results
            print(f"\n🔍 First 5 results:")
            print(results_df[['index', 'label', 'rating']].head())
            
        else:
            print("⚠️ Gemma classifier not available")
            print("💡 Generating simple output without ML classification...")
            
            # Simple output without classification
            simple_results = []
            for idx, row in sample_df.iterrows():
                simple_results.append({
                    'index': idx + 1,
                    'text': str(row['text']),
                    'rating': row.get('rating', 'N/A'),
                    'label': 'NOT_CLASSIFIED',
                    'reason': 'Classifier not available'
                })
            
            # Save simple results
            simple_df = pd.DataFrame(simple_results)
            output_filename = f"sample_200_reviews_no_classification_{int(time.time())}.csv"
            simple_df.to_csv(output_filename, index=False)
            print(f"💾 Simple results saved to: {output_filename}")

elif 'df' not in locals() or df.empty:
    print("⚠️ No dataset loaded yet")
    print("💡 Please run the data loading cells first to load your review dataset")

else:
    print("❌ Dataset not available")
    print("🔄 Please ensure dataset is properly loaded")

print("\n" + "=" * 70)

🤖 SAMPLE REVIEW CLASSIFICATION - 200 REVIEWS
🧪 Testing JSONL parsing on your review-other.json file...
✅ Successfully parsed sample data:
📊 Shape: (5, 8)
📋 Original columns: ['user_id', 'name', 'time', 'rating', 'text', 'pics', 'resp', 'gmap_id']
🧹 Applying Kaggle-style cleaning to DataFrame with columns: ['user_id', 'name', 'time', 'rating', 'text', 'pics', 'resp', 'gmap_id']
🧹 Cleaned dataset: 5 → 5 rows (removed 0)
✅ Cleaned sample:
📊 Shape: (5, 6)
📋 Columns: ['text', 'rating', 'business_name', 'author_name', 'photo', 'rating_category']
🔍 First few rows:
                                                text  rating    business_name  \
0  Andrea is amazing. Our dog loves her and she a...       5  Amber Thibeault   
1  Andrea does a wonderful  job  with our wild Pr...       5           Esther   
2                                  Never called back       1      Bob Barrett   

  author_name photo rating_category  
0      user_0  <NA>            <NA>  
1      user_1  <NA>            <NA>

## 🎯 Summary & Next Steps

### 🏆 What We've Built

Our **Trustworthy Location Review System** includes:

1. **🔍 Advanced Feature Engineering**
   - 40+ textual and non-textual features
   - Sentiment analysis, spam detection, readability metrics
   - Restaurant & Establishments relevancy indicators

2. **🚫 Rule-Based Policy Detection**
   - Advertisement detection (contact info, promotional language)
   - Irrelevant content filtering* (off-topic discussions)
   - Rant without visit identification* (hearsay reviews)

3. **🤖 Gemma 3 12B Integration**
   - Our LLM of choice for policy violation detection
   - Quality assessment and authenticity scoring
   - HuggingFace Inference Client integration

### 📊 System Capabilities

- **Policy Violation Detection**: Identifies advertisements, irrelevant content, and fake rants
- **Quality Assessment**: Evaluates review authenticity and helpfulness
- **Batch Processing**: Handles large datasets efficiently
- **Ensemble Decision Making**: Leverages multiple approaches for robust predictions
- **Explainable AI**: Provides reasoning for each classification decision

### 🚀 Production Readiness

The system is designed for:
- **Scalability**: Batch processing with configurable sizes
- **Reliability**: Fallback mechanisms when individual components fail
- **Flexibility**: Adjustable weights and thresholds
- **Monitoring**: Comprehensive performance analytics

### 📈 Next Steps

1. **🔧 Fine-Tuning**
   - Adjust ensemble weights based on validation data
   - Optimize decision thresholds
   - Add domain-specific rules

2. **📊 Evaluation**
   - Test on larger datasets
   - Measure precision, recall, F1-score
   - A/B test different configurations

3. **🚀 Further Improvements**
   - Find more "bad" examples to train the model better
   - Implement monitoring dashboard

### 🎉 Hackathon Impact

This solution addresses the critical problem of **review trustworthiness** by:
- **Filtering noise** from restaurant review platforms
- **Protecting consumers** from misleading information
- **Supporting businesses** with authentic feedback
- **Improving platform quality** through automated moderation